# Actividad 5 - Regresión Ordinal

Cargamos los datos del excel

In [ ]:
library(readxl)
if (.Platform$OS.type=="unix"){ ruta_excel <- "datasets/SPYO 1.xlsx"
} else ruta_excel <- "../datasets/SPYO 1.xlsx"

datos <- read_excel(ruta_excel, sheet="Sheet1")
head(datos, n=3)

Ordenamos nuestros datos:

In [ ]:
suppressPackageStartupMessages(library(dplyr))

# Ordenamos nuestros datos
datos <- datos %>% 
    arrange(spyo, edad, sexo, asiste, NS, tipo)

# SPYO es sobrepeso y obesidad
# NS es nivel socioeconómico

Creamos nuestras variables ordinales:

In [ ]:
datos$spyo <- factor(
    datos$spyo, levels=c("normal", "sobrepeso", "obesidad"),
    ordered=TRUE
)

datos$NS <- factor(
    datos$NS, levels=c("Bajo", "Medio", "Alto"),
    #ordered=TRUE los L/Q/C salen de model.matrix Linear/Quadratic/Cubic
)

datos$sexo <- as.factor(datos$sexo)

datos$asiste <- as.factor(datos$asiste)

datos$tipo <- as.factor(datos$tipo)

Creamos las tablas de contingencia para ver cómo están distribuidos los datos:

In [ ]:
addmargins(xtabs(formula = ~spyo+asiste, data=datos))

In [ ]:
addmargins(xtabs(formula = ~spyo+tipo, data=datos))

Hacemos un gráfico para ver cómo están distribuidos los datos por tipo:

In [ ]:
suppressPackageStartupMessages(library(ggplot2))

ggplot(datos, aes(spyo,fill=tipo)) + 
    geom_bar() + theme_minimal() + facet_wrap(.~tipo)

Calculamos nuestro modelo para la regresión ordinal:

In [ ]:
options(scipen=999) # Deshabilitar notación científica
suppressPackageStartupMessages(library(MASS))
library(broom)

imc_or <- polr(
    formula = spyo~edad+sexo+asiste+NS+tipo,
    data=datos[complete.cases(datos),], Hess=TRUE, method="logistic")
# Hay algunos asiste que tienen N/A. Los eliminamos.

Graficamos los efectos marginales del modelo para las variables edad, tipo y :

In [ ]:
library(effects)
plot(allEffects(imc_or), axes=list(y=list(style="stacked")))[1:3]

Calculamos los coeficientes exponenciados:

In [ ]:
tidy(imc_or, exponentiate = TRUE, conf.int = TRUE, conf.level = 0.95)

De la razón de momios se concluye que:

- La posibilidad de pasar de IMC normal a obesidad disminuye (muy poco) cuando el niñe es de sexo femenino a comparación de un niñe de sexo masculino. El aumento de estrato aumenta la posibilidad de obesidad, también se observa que aunenta la posibilidad de sobrepeso al pasar de un tipo de vivienda rural a urbana. En todos los casos la comparación se hace con las demás variables constantes mientras sólo se cambia una de ellas.

In [ ]:
car::Anova(imc_or, type=3)

Hacemos la prueba de hipótesis para los coeficientes de las variables:

$H_{0}$: $\beta_{1}=...=\beta_{n}= 0$

$H_{a}$: No existe $\beta_{n} = 0$

Nota: Como en el caso de la regresión ordinal el programa _no muestra los valores p_, utilizamos el intervalo de confianza para el estadístico:

- Las variables edad, NS y tipo son significativas porque su intervalo está sobre el 1, también no se puede concluir si hay asociación entre las demás variables, porque su intervalo de confianza está sobre el 0. En la prueba de anova también se ve que la aseveración anterior es se cumple.

# Actualización después de calificar la tarea:

  - Con respecto a todos tus cálculos, como corriste el modelo, como muestras las tablas muy bien, las gráficas cómo las demuestras excelente, sin embargo, creo que la conclusión te quedaste un poco corto en esta ocasión.

  - Si vemos en la parte de las gráficas de efectos que están en la hoja 3, podemos concluir varias cosas primero que, con respecto a la edad, conforme aumenta la edad disminuye la probabilidad de tener un peso normal, recuerda que son variables categóricas ordinales tienen un orden no puedes pasar de normal a obesidad sin considerar sobrepeso y en este caso se va se va calculando o acumulando la probabilidad.

  - Después parece ser que conforme aumenta la edad si aumenta la probabilidad de tener sobrepeso o el cambio de normal a sobrepeso y en el caso de obesidad no se nota o no es tan significativo el cambio de probabilidad, aunque también aumenta la probabilidad de tener obesidad.

  - Con respecto a lo que es el sexo no se ve ningún efecto,  en la parte de asiste el nivel socioeconómico si se logra ver un cierto cambio en el nivel socioeconómico sí se ve un efecto en cuanto a que en el nivel socioeconómico alto disminuye la probabilidad de tener un peso normal y aumenta la probabilidad de tener sobrepeso así también aumenta la probabilidad de tener obesidad y con respecto al tipo si es rural o urbano pues también se ve significativo que en la zona urbana disminuye la probabilidad de tener un peso normal y aumenta la probabilidad de tener un sobrepeso.

  - Aquí creo que te faltó también considerar las interacciones y en este ejercicio sí había una interacción significativa, recuerda que siempre debemos de partir del modelo saturado, no veo que lo menciones, pero a partir del modelo saturado podemos verificar normalmente en este tipo de ejemplo al menos una interacción resulta significativa